In [ ]:
import pandas as pd
import numpy as np
import time
import torchvision
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm

from PIL import Image, ImageFile
import cv2
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler
import os
import albumentations
from albumentations import torch as AT
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR


device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset
from torchvision import transforms

import matplotlib.pyplot as plt


In [ ]:
device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [ ]:
def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [ ]:
def circle_crop(img, sigmaX=10):   
    """
    Create circular crop around image centre    
    """    
    
    img = cv2.imread(img)
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted ( img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 

In [ ]:
class RetinopathyDatasetTrain(Dataset):
    
    def __init__(self, csv_file, transform=None):
        self.data = csv_file
        self.transform = transform
        self.image_files_list = [f'../input/aptos2019-blindness-detection/train_images/{i}.png' for i in self.data['id_code'].values]
        
    def __len__(self):
        return len(self.image_files_list)

    def __getitem__(self, idx):
        img_name = self.image_files_list[idx]
        #img = cv2.imread(img_name)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = circle_crop(img_name)
        image = self.transform(image=img)
        image = image['image']
        label = self.data.loc[idx, 'diagnosis']
        return image, label

        
        
        

In [ ]:
class RetinopathyDatasetTest(Dataset):
    
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.image_files_list = [f'../input/aptos2019-blindness-detection/test_images/{i}.png' for i in self.data['id_code'].values]
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.image_files_list[idx]
        #img = cv2.imread(img_name)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = circle_crop(img_name)
        image = self.transform(image=img)
        image = image['image']
        return image

In [ ]:
transforms = albumentations.Compose([
    albumentations.Resize(224, 224),
    albumentations.HorizontalFlip(),
    albumentations.RandomBrightness(),
    albumentations.ShiftScaleRotate(rotate_limit=15, scale_limit=0.10),
    albumentations.JpegCompression(80),
    albumentations.HueSaturationValue(),
    albumentations.Normalize(),
    AT.ToTensor()
    ])
transforms_test = albumentations.Compose([
    albumentations.Resize(224, 224),
    albumentations.Normalize(),
    AT.ToTensor()
    ])


In [ ]:
model = torchvision.models.resnet101(pretrained=False)
model.load_state_dict(torch.load("../input/pytorch-pretrained-models/resnet101-5d3b4d8f.pth"))
num_features = model.fc.in_features
model.fc = nn.Linear(2048, 1)

model = model.to(device)

In [ ]:
train = pd.read_csv("/kaggle/input/aptos2019-blindness-detection/train.csv")
dataset = RetinopathyDatasetTrain(csv_file = train, transform = transforms)

tr, val = train_test_split(train.diagnosis, stratify=train.diagnosis, test_size=0.1)
train_sampler = SubsetRandomSampler(list(tr.index))
valid_sampler = SubsetRandomSampler(list(val.index))
batch_size = 16
num_workers = 4
# prepare data loaders (combine dataset and sampler)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
validloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)


"""plist = [
         {'params': model.layer4.parameters(), 'lr': 1e-4, 'weight': 0.001},
         {'params': model.fc.parameters(), 'lr': 1e-3}
         ]
"""

criterion = nn.BCEWithLogitsLoss()
criterion_2 = nn.MSELoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.99)
optimizer_2 = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10)

In [ ]:
valid_loss_min = np.Inf
criterion = nn.MSELoss()
patience = 5
# current number of epochs, where validation loss didn't increase
p = 0
# whether training should be stopped
stop = False

# number of epochs to train the model
n_epochs = 2
for epoch in range(1, n_epochs+1):
    print(time.ctime(), 'Epoch:', epoch)

    train_loss = []
    train_auc = []

    for i, (data, target) in enumerate(dataloader):

        data, target = data.cuda(), target.cuda().view(-1,1)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.float())
        train_loss.append(loss.item())        
        #a = target.data.cpu().numpy()
        #b = output[:,-1].detach().cpu().numpy()
        # train_auc.append(roc_auc_score(a, b))
        loss.backward()
        optimizer.step()
        print("train step")
    
    model.eval()
    val_loss = []
    val_auc = []
    for i, (data, target) in enumerate(validloader):
        data, target = data.cuda(), target.cuda().view(-1,1)
        output = model(data)
        loss = criterion(output, target.float())
        val_loss.append(loss.item()) 
        print("val step")
        #a = target.data.cpu().numpy()
        #b = output[:,-1].detach().cpu().numpy()
        # val_auc.append(roc_auc_score(a, b))

    # print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}, train auc: {np.mean(train_auc):.4f}, valid auc: {np.mean(val_auc):.4f}')
    print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}.')
    
    valid_loss = np.mean(val_loss)
    scheduler.step(valid_loss)
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss
        p = 0

    # check if validation loss didn't improve
    if valid_loss > valid_loss_min:
        p += 1
        print(f'{p} epochs of increasing val loss')
        if p > patience:
            print('Stopping training')
            stop = True
            break        
            
    if stop:
        break

In [ ]:
torch.save({'arch': 'resnet101',
            'state_dict': model.state_dict(), 
            'class_to_idx': model.class_to_idx}, 
            'classifier.pth')

In [ ]:
"""since = time.time()

criterion = nn.MSELoss()
num_epochs = 15
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    scheduler.step()
    model.train()
    running_loss = 0.0
    tk0 = tqdm(dataloader, total=int(len(dataloader)))
    counter = 0
    for bi, (inputs, labels) in enumerate(tk0):
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float).view(-1,1)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        counter += 1
        tk0.set_postfix(loss=(running_loss / (counter * dataloader.batch_size)))
    epoch_loss = running_loss / len(dataloader)
    print('Training Loss: {:.4f}'.format(epoch_loss))

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
torch.save(model.state_dict(), "model.bin")"""
        

In [ ]:
test_dataset = RetinopathyDatasetTest(csv_file = "/kaggle/input/aptos2019-blindness-detection/test.csv", transform = transforms_test)


In [ ]:
def tta_func(model):
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)
    with torch.no_grad():
        predict = np.zeros((len(test_dataset), 1))

        for i, image in enumerate(test_dataloader):
            pred = model(image.to(device))   
            predict[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)
    return predict

In [ ]:
predict1 = tta_func(model)
predict2 = tta_func(model)
predict3 = tta_func(model)
predict4 = tta_func(model)
predict5 = tta_func(model)
predict6 = tta_func(model)
predict7 = tta_func(model)
predict8 = tta_func(model)
predict9 = tta_func(model)
predict10 = tta_func(model)

In [ ]:
test_preds = (predict1 + predict2 + predict3 + predict4 + predict5
             + predict6 + predict7 + predict8+ predict9 + predict10) /10.0

In [ ]:

coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(test_preds):
    if pred < coef[0]:
        test_preds[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        test_preds[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        test_preds[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        test_preds[i] = 3
    else:
        test_preds[i] = 4



In [ ]:

sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = test_preds.astype(int)
sample.to_csv("submission.csv", index=False)